In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper

### Exercise 1

Find the uncompressed SEC format for the Public Key where the Private Key secrets are:

* 5000
* \\(2018^{5}\\)
* 0xdeadbeef12345

In [15]:
"""SEC : Standards for Efficient Cryptography

Standard for ECDSA public keys, comes in compressed and uncompressed formats.
See https://secg.org/sec1-v2.pdf#subsubsection.2.3.3

Uncompressed SEC can be encoded like so:
    1. Start with prefix byte 0x04 (acts as a signifier of what to expect)
    2. Append the pubkey x coordinate (32 bytes big endian)
    3. Append the pubkey y coordinate (32 bytes big endian)

"""
# Exercise 1
from ecc import PrivateKey

secrets = (5000, 2018**5, 0xdeadbeef12345)
def UncompSECEncode(pub_key):
    return b'\x04' + pub_key.x.num.to_bytes(32, 'big') + pub_key.y.num.to_bytes(32, 'big')

for secret in secrets:
    print(UncompSECEncode(PrivateKey(secret).point).hex())
print()
# Using Song's S256Point method sec:
for secret in secrets:
    print(PrivateKey(secret).point.sec(compressed=False).hex())

04ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c315dc72890a4f10a1481c031b03b351b0dc79901ca18a00cf009dbdb157a1d10
04027f3da1918455e03c46f659266a1bb5204e959db7364d2f473bdf8f0a13cc9dff87647fd023c13b4a4994f17691895806e1b40b57f4fd22581a4f46851f3b06
04d90cd625ee87dd38656dd95cf79f65f60f7273b67d3096e68bd81e4f5342691f842efa762fd59961d0e99803c61edba8b3e3f7dc3a341836f97733aebf987121

04ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c315dc72890a4f10a1481c031b03b351b0dc79901ca18a00cf009dbdb157a1d10
04027f3da1918455e03c46f659266a1bb5204e959db7364d2f473bdf8f0a13cc9dff87647fd023c13b4a4994f17691895806e1b40b57f4fd22581a4f46851f3b06
04d90cd625ee87dd38656dd95cf79f65f60f7273b67d3096e68bd81e4f5342691f842efa762fd59961d0e99803c61edba8b3e3f7dc3a341836f97733aebf987121


### Exercise 2

Find the Compressed SEC format for the Public Key where the Private Key secrets are:

* 5001
* \\(2019^{5}\\)
* 0xdeadbeef54321

In [88]:
"""Compressed SEC

The SEC format can be compressed by leveraging some characteristics of elliptic curves and finite
fields. If we know x, y can be computed with the elliptic curve formula, or y**2 == x**3 + a*x + b.
Also, for any given x, there can only be one y value, or in the case of the intersection of a
vertical line not tangent to a real number elliptic curve, two y values, y and -y.

However, since this is over a finite field, instead of y and -y, we could consider it y % P and
(P - y) % P. Differentiating these two possibilites now becomes a matter of evenness rather than
sign. P must be a prime greater than 2, and so is odd*. So if y is even, p-y is odd, and if y is odd,
p-y is even. In other words, in a elliptic curve accross real numbers, it could be (x,y) or (x,-y),
but for curves accross a finite field, it's (x, (even y satisfying curve equation)) or
(x, (odd y satisfying curve equation)).

So, only the x coordinate and the sign of the y are truly needed when data size is a consideration.
Compressed SEC is encoded like so:
    1. Prefix byte: 0x02 if y is even, 0x03 if y is odd
    2. Append pubkey x coordinate in 32-byte big endian

* In this section of the book, Song asserts that P for a given elliptic curve over a finite field
must be greater than 2. Should we establish that as a class invariant in FieldElement? 

"""
# Exercise 2
from ecc import PrivateKey

secrets = (5000, 2018**5, 0xdeadbeef12345)
def CompSECEncode(pub_key):
    prefix = b'\x02' if pub_key.y.num % 2 == 0 else b'\x03'
    return prefix + pub_key.x.num.to_bytes(32, 'big')

for secret in secrets:
    print(CompSECEncode(PrivateKey(secret).point).hex())
print()
# Using Song's S256Point.sec()
for secret in secrets:
    # implicit sec(compressed=True)
    print(PrivateKey(secret).point.sec().hex())

"""Decoding SEC

See https://secg.org/sec1-v2.pdf#subsubsection.2.3.4

Decoding requires deriving x from y, which for y**2 = x**3 + ax + b means that we need
to take the square root of a finite field element. One of the characteristics of secp256k1
is that P % 4 = 3, which can be used here to help derive an answer:

Because P % 4 = 3, (P + 1) % 4 = 0, so (P + 1)/4 is an integer.

Looking for the square root, or w**2 = v, can be transformed via Fermat's Little Theorem, to
w**(P - 1) % P = 1, and so w**2 = w**2 * 1 = w**2 * w**(P - 1) = w**(P + 1).

Any prime other than 2 / 2 % P should equal an integer, so for w**2 = w**(P + 1) we
can divide both exponents by 2 to get w = w**(P + 1)/2.

Further, if (P + 1)/4 is an integer, then w = w**(P + 1)/2 = w**2(P + 1)/4 =
(w**2)**(P + 1)/4 = v**(P + 1)/4, or w = v**(P + 1)/4 if P % 4 = 3

Recall:
Uncompressed SEC can be encoded like so:
    1. Start with prefix byte 0x04 (acts as a signifier of what to expect)
    2. Append the pubkey x coordinate (32 bytes big endian)
    3. Append the pubkey y coordinate (32 bytes big endian)
Compressed SEC is encoded like so:
    1. Prefix byte: 0x02 if y is even, 0x03 if y is odd
    2. Append pubkey x coordinate in 32-byte big endian

    Song's PrivateKey.parse()

"""
from ecc import P, B, S256Field, S256Point

"""
# S256Point.sqrt
def sqrt(self):
    # floor div to force int exponent
    return self**(P + 1) // 4
"""
def decode_sec(sec):
    compressed = True
    if sec[0] == 4:  # b'\x04'
        compressed = False
    elif sec[0] == 2:  # b'\x02'
        even_y = True
    elif sec[0] == 3:  # b'\x03'
        even_y = False
    else:
        raise ValueError("Invalid SEC prefix byte of {:x}".format(sec[0]))
    x = S256Field(int.from_bytes(sec[1:33], 'big'))
    if not compressed:
        return S256Point(x=x, y=S256Field(int.from_bytes(sec[33:65], 'big')))
    # y**2 = x**3 + 7
    alpha = x**3 + S256Field(B)  # B == 7
    beta = alpha.sqrt()
    if (beta.num % 2 == 0):
        even_beta = beta
        odd_beta = S256Field(P - beta.num)
    else:
        even_beta = S256Field(P - beta.num)
        odd_beta = beta
    if even_y:
        return S256Point(x, even_beta)
    else:
        return S256Point(x, odd_beta)
    
for secret in secrets:
    print(decode_sec(PrivateKey(secret).point.sec()) ==
          S256Point.parse(PrivateKey(secret).point.sec()))
        

02ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c
02027f3da1918455e03c46f659266a1bb5204e959db7364d2f473bdf8f0a13cc9d
03d90cd625ee87dd38656dd95cf79f65f60f7273b67d3096e68bd81e4f5342691f

02ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c
02027f3da1918455e03c46f659266a1bb5204e959db7364d2f473bdf8f0a13cc9d
03d90cd625ee87dd38656dd95cf79f65f60f7273b67d3096e68bd81e4f5342691f
True
True
True


### Exercise 3

Find the DER format for a signature whose `r` and `s` values are:

* r =

`0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6`

* s =

`0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec`

In [42]:
"""DER - Distinguished Encoding Rules

DER, likely taken from early Bitcoin implementation's use of OpenSSL, is
used to (de)serialize signatures.

Both r and s need to be encoded, as s cannot solely be derived from r.

DER is as follows:
    1. Prefix of 0x30
    2. Length of the encoded signature to follow, in bytes (single byte, endianness not relevant)
    3. Marker byte 0x02
    4. r in big-endian, with leading 0x00 bytes trimmed, and then one added back if first non 0x00
        byte is >= 0x80, to prevent being seen as negative. Prepend length of resulting value in
        bytes.
    5. Marker byte 0x02
    6. s in big-endian, with leading 0x00 bytes trimmed, and then one added back if first non 0x00
        byte is >= 0x80, to prevent being seen as negative. Prepend length of resulting value in
        bytes.

"""
# Exercise 3

from ecc import Signature

r = 0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6
s = 0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec

def PrepIntForDER(value):
    bin = value.to_bytes(32, 'big')
    # remove leading null bytes
    bin.lstrip(b'\x00')
    # restore single leading null byte if value could be interpreted as negative
    if bin[0] >= 0x80:
        bin = b'\x00' + bin
    return b'\x02' + len(bin).to_bytes(1, 'big') + bin

r_der = PrepIntForDER(r)
s_der = PrepIntForDER(s)
print((b'\x30' + (len(r_der) + len(s_der)).to_bytes(1, 'big') + r_der + s_der).hex())

# Song's Signature method der:
sig = Signature(r=r, s=s)
print(sig.der().hex())
print(sig.der() == b'\x30' + (len(r_der) + len(s_der)).to_bytes(1, 'big') + r_der + s_der)


3045022037206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c60221008ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec
3045022037206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c60221008ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec
True


### Exercise 4

Convert the following hex to binary and then to Base58:

* `7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d`
* `eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c`
* `c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6`

In [52]:
"""Base 58 notation - all digits + upper alpha + lower alpha - '0' - '1' - 'O' - 'l'

According to Song, may eventually be replaced by the Bech32 standard used now in Segwit:
digits + lower alpha - '1' -'b' - 'i' - 'o'.

"""

# Exercise 4

hex_strs = ("00000001",
            "7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d",
            "eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c",
            "c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6")

# digits + caps + lower - (0, 1, O, l) 
BASE58_CHARSET =  "123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz"

def base58(bytes_obj):
    # manually transcode leading null bytes (for fixed-width bytes objects)
    null_byte_ct = 0
    for byte in bytes_obj:
        if byte == 0x00:
            null_byte_ct += 1
        else:
            break
    prefix = BASE58_CHARSET[0x00] * null_byte_ct
    num = int.from_bytes(bytes_obj, 'big')
    b58_str = ""
    while num > 0:
        num, mod = divmod(num, 58)
        b58_str = BASE58_CHARSET[mod] + b58_str
    return prefix + b58_str

for hex_s in hex_strs:
    print(base58(bytes.fromhex(hex_s)))
print()

# Using Song's method:
from helper import encode_base58
# test of leading ones
for hex_s in hex_strs:
    print(encode_base58(bytes.fromhex(hex_s)))
    print(base58(bytes.fromhex(hex_s)) == encode_base58(bytes.fromhex(hex_s)))


1112
9MA8fRQrT4u8Zj8ZRd6MAiiyaxb2Y1CMpvVkHQu5hVM6
4fE3H2E6XMp4SsxtwinF7w9a34ooUrwWe4WsW1458Pd
EQJsjkd6JaGwxrjEhfeqPenqHwrBmPQZjJGNSCHBkcF7

1112
True
9MA8fRQrT4u8Zj8ZRd6MAiiyaxb2Y1CMpvVkHQu5hVM6
True
4fE3H2E6XMp4SsxtwinF7w9a34ooUrwWe4WsW1458Pd
True
EQJsjkd6JaGwxrjEhfeqPenqHwrBmPQZjJGNSCHBkcF7
True


### Exercise 5

Find the address corresponding to Public Keys whose Private Key secrets are:

* 5002 (use uncompressed SEC, on testnet)
* \\(2020^{5}\\) (use compressed SEC, on testnet)
* 0x12345deadbeef (use compressed SEC on mainnet)

In [60]:
"""Bitcoin address format - shortened and obfuscated form of public key used as wallet
addresses

Bitcoin addresses are serialized as follows:
    1. For mainnet prefix 0x00, testnet 0x6f
    2. Create a hash160 of the public key (SEC of key (compressed or not*) > sha256 > ripemd160)
    3. Combine #1 and #2
    4. Take first four bytes of hash256 (sha256 twice) of #3
    5. Encode #3 and #4 in base58

* TBD: Song states that there is the choice of compressed or uncompressed SEC - does this mean that
there are four addresses avaialable for every public key? (mainnet short and long SEC, and testnet
short and long SEC)?

"""

# Exercise 5

from ecc import PrivateKey
import hashlib
sha256 = hashlib.sha256
from helper import encode_base58
# 5002 (use uncompressed SEC, on testnet)
# 2020**5 (use compressed SEC, on testnet)
# 0x12345deadbeef (use compressed SEC on mainnet)
test_inputs = ((5002, True), (2020**5, True), (0x12345deadbeef, False))

def encode_address(pub_key, testnet=True):
    prefix = b'\x6f' if testnet else b'\x00'
    h160 = hashlib.new('ripemd160', sha256(pub_key.sec()).digest()).digest()
    h160 = prefix + h160
    h256 = sha256(sha256(h160).digest()).digest()
    return encode_base58(h160 + h256[:4])

for secret, testnet in test_inputs:
    print(encode_address(PrivateKey(secret).point, testnet))
    # Comparing to Song's S256Point.address()
    print(encode_address(PrivateKey(secret).point, testnet) ==
             PrivateKey(secret).point.address(testnet=testnet))
    

mqsBj1baxzgPTeeRJbE8cbgdYmtc3yess3
True
mopVkxp8UhXqRYbCYJsbeE1h1fiF64jcoH
True
1F1Pn2y6pDb68E5nYJJeba4TLg2U7B6KF1
True


### Exercise 6

Find the WIF for Private Key whose secrets are:

* 5003 (compressed, testnet)
* \\(2021^{5}\\) (uncompressed, testnet)
* 0x54321deadbeef (compressed, mainnet)

In [70]:
"""WIF - Wallet Import Format

In the comparatively rare cases where a private key needs to be communicated, we can use WIF
to serialize it like so:
    1. Prefix of 0x80 for mainnet, 0xef for testnet
    2. Secret encoded in 32 byte big endian
    3. If SEC for public key was compressed, use suffix 0x01*
    4. Combine #1, #2, and #3
    5. Take first 4 bytes of hash256 (sha256 twice) of #4
    6. Encode #4 and #5 in base 58.

* TBD: This implies that any use of WIF also requires not only the inclusion, but the
fresh reencoding, of an address, as we need to know the address's SEC size.

"""

# Exercise 6

from ecc import PrivateKey
from hashlib import sha256
from helper import encode_base58
test_inputs = ((5002, True, True), (2020**5, False, True), (0x12345deadbeef, False, False))

def encode_wif(pri_key, testnet, sec_compressed):
    prefix = b'\xef' if testnet else b'\x80'
    secret_bin = secret.to_bytes(32, 'big')
    suffix = b'\x01' if sec_compressed else b''
    h256 = sha256(sha256(prefix + secret_bin + suffix).digest()).digest()
    return encode_base58(prefix + secret_bin + suffix + h256[:4])
    
for secret, compressed_sec, testnet in test_inputs:
    print(encode_wif(secret, testnet, compressed_sec))
    print(encode_wif(secret, testnet, compressed_sec) ==
        PrivateKey(secret).wif(testnet=testnet, compressed=compressed_sec))

cMahea7zqjxrtgAbB7LSGbcQUr1uX1ojuat9jZodMN8rEy143Tdw
True
91avARGdfge8E4tZfYLoxeJ5sGBdNJQH4kvjpRuzvCbAb45H4xp
True
5HpHagT65TZzG1PH3CSu63k8DbpvD8s5ip4nETQsYsXAmH5Pk3h
True


### Exercise 7

Write a function `little_endian_to_int` which takes Python bytes, interprets those bytes in Little-Endian and returns the number.

#### Make [this test](/edit/code-ch04/helper.py) pass: `helper.py:HelperTest:test_little_endian_to_int`

In [73]:
# Exercise 7

reload(helper)
run(helper.HelperTest("test_little_endian_to_int"))

.
----------------------------------------------------------------------
Ran 1 test in 0.005s

OK


### Exercise 8

Write a function `int_to_little_endian` which does the reverse of the last exercise.

#### Make [this test](/edit/code-ch04/helper.py) pass: `helper.py:HelperTest:test_int_to_little_endian`

In [74]:
# Exercise 8

reload(helper)
run(helper.HelperTest("test_int_to_little_endian"))

.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


### Exercise 9

Create a testnet address for yourself using a long secret that only you know. This is important as there are bots on testnet trying to steal testnet coins. Make sure you write this secret down somewhere! You will be using the secret later to sign Transactions.

In [77]:
# Exercise 9

from ecc import PrivateKey
from helper import hash256, little_endian_to_int

# select a passphrase here, add your email address into the passphrase for security
# passphrase = b'your@email.address some secret only you know'
passphrase = b'placeholder'
secret = little_endian_to_int(hash256(passphrase))
# create a private key using your secret
# print an address from the public point of the private key with testnet=True
print(PrivateKey(secret).point.address(testnet=True))

mjurPTqUces4EmN21g1tR7JadSrqyFmfY8
